# GMRES

In [77]:
using LinearAlgebra
using SparseArrays
using Distributions
using PyCall
using Statistics
using BenchmarkTools

@pyimport numpy 

function gmres(A, B, X0, max_iter)
    r = B .- A * X0
    x = Array{Float64, 2}[]
    q = Array{Any}(zeros(max_iter)) #array any, gdyż przechowuje i wektory i pojedyncze liczby 
    q[1] = r/norm(r)
    h = zeros((max_iter + 1, max_iter))
    
    for m=1:max_iter
        y = A * q[m]
        
        for j=1:m-1
            h[j, m] = (q[j]' * y)[1]
            y = y - h[j, m] * q[j] 
        end
        
        h[m + 1, m] = norm(y)
        
        if (h[m + 1, m] != 0) && (m != max_iter)
            q[m + 1] = y / h[m + 1, m]
        end

        b = zeros(max_iter + 1)
        b[1] = norm(r)
        result = (h \ b)

        if m > max_iter - 2
            z = reduce(hcat, q[:])
        else
            z = q
        end

        append!(x, [numpy.dot(z, result)+X0])
    end
    
    return x[max_iter]
end


gmres (generic function with 1 method)

In [122]:
function restarted_gmres(A, B, X0, max_iter, res)
    new_x0 = gmres(A, B, X0, res)
    for i=2*res:res:max_iter
        new_x0 = gmres(A, B, new_x0, res)
    end
    return new_x0
end

restarted_gmres (generic function with 1 method)

In [79]:
#generacja macierzy A oraz wektorów B i X0
A = Symmetric(sprand(40,40,0.5))
B = rand(Uniform(0,1), 40, 1)
X0 = rand(Uniform(0,1), 40, 1)

40×1 Matrix{Float64}:
 0.7606391431179831
 0.39117343244311686
 0.46026923116094154
 0.29683241626620904
 0.002773703427796148
 0.7191563896885242
 0.22473453380192843
 0.215108054220845
 0.026243794078913085
 0.05336168029597621
 0.5789618792355196
 0.6858960060470822
 0.9040354101287693
 ⋮
 0.36470026961515334
 0.4148052417754269
 0.276141367183105
 0.2904156259725763
 0.5787885192262106
 0.6942110928738621
 0.856924086639272
 0.0369590897190486
 0.978214805287261
 0.4203278026011259
 0.571966613074075
 0.9335881859763671

In [166]:
#ustalenie parametrów początkowych
max_iter = 300
res = 100

100

In [167]:
#uruchomienie bazowego gmres
@btime x1 = gmres(A, B, X0, max_iter)

  3.079 s (880164 allocations: 551.34 MiB)


40×1 Matrix{Float64}:
 -0.23204846819451797
 -8.308101367080049
 -2.567115471194662
  1.4508256369342507
  0.4831340884389743
 -4.164164762719872
  3.8632384487738483
  2.6840618379650496
 -2.8874337230338574
 -1.1157877064153898
  0.5034822825095557
  0.37794828944424896
 -1.599963048148649
  ⋮
  4.50823491731588
 -3.9779821139120557
  2.2049374616721344
  1.5586441408086704
 -0.6193247677578106
  4.163307412931417
  0.8833702080808207
  4.947285551984006
  0.12427055400593101
  0.010667145736592332
 -1.1105176686001983
  3.3775810351552003

In [168]:
#uruchomienie restarted gmres
@btime x2 = restarted_gmres(A, B, X0, max_iter*10, res)

  3.036 s (3088175 allocations: 1.11 GiB)


40×1 Matrix{Float64}:
 -0.28540462757055657
 -8.102495879776638
 -2.453003343224105
  1.419308479527588
  0.5084585608950501
 -4.033054599013598
  3.725406033193327
  2.5557361944363515
 -2.881989628385638
 -1.0559358265819117
  0.5081395805858652
  0.3210887879926558
 -1.5530126375830968
  ⋮
  4.380354581793889
 -3.8765806226586905
  2.221376425361447
  1.5086395644035393
 -0.5651573373182394
  3.995037434937755
  0.8643909365585933
  4.834778576252543
  0.1481663967369528
 -0.006967589132115174
 -1.1487776544928883
  3.2423494685363874

In [169]:
#julia linear system solver
x3 = A\B

40×1 Matrix{Float64}:
 -0.25532278236600947
 -8.198219122561914
 -2.5113421107964546
  1.434249642264954
  0.49184416738822717
 -4.096740594700578
  3.793049525608861
  2.6261453449783696
 -2.8776861685827737
 -1.0885906250514517
  0.500397443577207
  0.35423012899185974
 -1.5753573211032137
  ⋮
  4.444602106411869
 -3.9234404956431406
  2.202711403072487
  1.5336807858085424
 -0.5936964952275787
  4.080017202018842
  0.8762546022177471
  4.884134748666695
  0.13406529528867334
  0.0020413069147766872
 -1.1244080195620627
  3.311790010828837

In [170]:
#absolute error
abs_res = broadcast(abs, x2-x3)
abs_normal = broadcast(abs, x1-x3)

mean_abs_res = mean(abs_res)
mean_abs_normal = mean(abs_normal)

println("restarted:")
println(mean_abs_res)
println("base:")
println(mean_abs_normal)

restarted:
0.0034461255578947885
base:
2.4262008063652113e-6


In [171]:
#relative error (percentage)
rel_res = abs_res./x3 * 100
rel_normal = abs_normal./x3 * 100

mean_rel_res = abs(mean(rel_res))
mean_rel_normal = abs(mean(rel_normal))

println("restarted:")
println(mean_rel_res)
println("base:")
println(mean_rel_normal)

restarted:
1.1366593399405238
base:
0.000813883139218001


# BICGSTAB

In [172]:
# linear system to solve
A = [1.0 1.0 1.0; 0 2.0 5.0; 2.0 5.0 -1.0]
B = [6.0; -4.0; 27.0]

3-element Vector{Float64}:
  6.0
 -4.0
 27.0

In [173]:
# starting configuration
iterations = 100
x0 = [0.0; 0.0; 0.0]
r0 = B - A*x0
r00 = r0
rho0 = 1.0
alpha = 1.0
omega0 = 1.0
v0 = 0.0
p0 = 0.0
rho=0.0

0.0

In [174]:
function bicgstab(A, B, x0, iterations)
    r0 = B - A*x0
    r00 = r0
    rho0 = 1.0
    alpha = 1.0
    omega0 = 1.0
    v0 = 0.0
    p0 = 0.0
    rho=0.0
    for iter = 1:iterations
        println(r0)
        rho = r00*r0'
        beta = (rho/rho0) * (alpha/omega0)
#     p = r0 + beta * (p0 - omega0 * v0)
        p = [r0;(beta * (p0 - omega0 * v0))]  # to pewnie źle
        v = A .* p
        alpha = rho/dot(r00,v)
        h = x0 + alpha*p
    # tu powinno być sprawdzanie dokładności h i jak będzie dobra to wyjdź z pętli
        s = r0 - alpha*v
        t = A .* s
        omega = dot(t, s) / dot(t, t)
        x = h + omega*s
    # tu powinno być sprawdzanie dokładności x i jak będzie dobra to wyjdź z pętli
        r = s - omega*t
    end
    return x
end

bicgstab (generic function with 1 method)